In [1]:
%%capture

# Upgrade pip
!pip install --upgrade pip
# Connectivity
!pip install psycopg2-binary  # PostgreSQL adapter
# !pip install snowflake-connector-python  # Snowflake connector
!pip install snowflake-connector-python==3.15.0 # Snowflake connector Older Version
!pip install snowflake-sqlalchemy  # Snowflake SQLAlchemy connector
!pip install warnings # Warnings management
# !pip install pyarrow # Serialization
!pip install keyring==23.11.0 # Key management
!pip install sqlalchemy==1.4.46 # SQLAlchemy
!pip install requests # HTTP requests
!pip install boto3 # AWS SDK
# !pip install slackclient # Slack API
!pip install oauth2client # Google Sheets API
!pip install gspread==5.9.0 # Google Sheets API
!pip install gspread_dataframe # Google Sheets API
!pip install google.cloud # Google Cloud
# Data manipulation and analysis
!pip install polars
!pip install pandas==2.2.1
!pip install numpy
# !pip install fastparquet
!pip install openpyxl # Excel file handling
!pip install xlsxwriter # Excel file handling
# Linear programming
!pip install pulp
# Date and time handling
!pip install --upgrade datetime
!pip install python-time
!pip install --upgrade pytz
# Progress bar
!pip install tqdm
# Database data types
!pip install db-dtypes
# Geospatial data handling
# !pip install geopandas
# !pip install shapely
# !pip install fiona
# !pip install haversine
# Plotting

# Modeling
!pip install statsmodels
!pip install scikit-learn

!pip install import-ipynb

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import calendar
import json
from datetime import date, timedelta
from oauth2client.service_account import ServiceAccountCredentials
import setup_environment_2
import importlib
import import_ipynb
import warnings
from datetime import datetime, timedelta
import pytz  
import os
import snowflake.connector
import boto3
warnings.filterwarnings("ignore")
importlib.reload(setup_environment_2)
setup_environment_2.initialize_env()
import base64
from botocore.exceptions import ClientError
from requests import get
from pathlib import Path
import requests
import time
import gspread

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (22.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json


In [3]:
def query_snowflake(query, columns=[]):
    import os
    import snowflake.connector
    import numpy as np
    import pandas as pd
    con = snowflake.connector.connect(
        user =  os.environ["SNOWFLAKE_USERNAME"],
        account= os.environ["SNOWFLAKE_ACCOUNT"],
        password= os.environ["SNOWFLAKE_PASSWORD"],
        database =os.environ["SNOWFLAKE_DATABASE"]
    )
    try:
        cur = con.cursor()
        cur.execute("USE WAREHOUSE COMPUTE_WH")
        cur.execute(query)
        if len(columns) == 0:
            out = pd.DataFrame(np.array(cur.fetchall()))
        else:
            out = pd.DataFrame(np.array(cur.fetchall()),columns=columns)
        return out
    except Exception as e:
        print("Error: ", e)
    finally:
        cur.close()
        con.close()

In [4]:
query = '''
WITH prs AS (
SELECT DISTINCT product_purchased_receipts.purchased_receipt_id,
                purchased_receipts.purchased_order_id,
                DATE_PART('Day', purchased_receipts.date::date) AS DAY,
                DATE_PART('month', purchased_receipts.date::date) AS MONTH,
                DATE_Part('year', purchased_receipts.date::date) AS YEAR,
				DATE_Part('hour', purchased_receipts.date) AS hour,
                products.id AS product_id,
                CONCAT(products.name_ar, ' ', products.size, ' ', product_units.name_ar) AS sku,
                brands.name_ar AS Brand,
                categories.name_ar as category,
                products.description,
                purchased_receipts.warehouse_id AS warehouse_id,
                warehouses.name as warehouse,
                packing_units.name_ar AS packing_unit,
                purchased_receipts.discount AS Total_discount,
                purchased_receipts.return_orders_discount,
                purchased_receipts.discount_type_id,
                suppliers.id AS supplier_id,
                suppliers.name AS supplier_name,
                purchased_receipt_statuses.name_ar AS PR_status,
                product_purchased_receipts.basic_unit_count,
                product_purchased_receipts.purchased_item_count AS purchase_count,
                product_purchased_receipts.purchased_item_count*product_purchased_receipts.basic_unit_count AS purchase_min_count,
                product_purchased_receipts.item_price,
                product_purchased_receipts.final_price/product_purchased_receipts.purchased_item_count AS final_item_price,
                product_purchased_receipts.total_price AS purchase_price,
                CASE WHEN product_purchased_receipts.vat = 'true' THEN product_purchased_receipts.total_price * 0.14
                     ELSE CASE WHEN product_purchased_receipts.vat = 'false' THEN product_purchased_receipts.total_price * 0
                               END
                END AS vat,
                CASE WHEN purchased_receipts.discount_type_id = 2 THEN (product_purchased_receipts.discount/100) * product_purchased_receipts.total_price
                     ELSE product_purchased_receipts.discount
                END AS SKU_discount,
                purchased_receipts.total_price AS pr_value,
                CASE
                    WHEN product_purchased_receipts.t_tax_id = 1 THEN product_purchased_receipts.total_price * 0.05
                    ELSE CASE
                             WHEN product_purchased_receipts.t_tax_id = 2 THEN product_purchased_receipts.total_price * 0.08
                             ELSE CASE
                                      WHEN product_purchased_receipts.t_tax_id = 3 THEN product_purchased_receipts.total_price * 0.1
                                      ELSE 0
                                  END
                         END
                END AS table_tax,
                product_purchased_receipts.final_price AS Final_Price,
                product_purchased_receipts.product_type_id,
                purchased_receipts.debt_note_value as credit_note,
                purchased_receipts.tips,
                purchased_receipts.delivery_fees,
                case when purchased_receipts.is_actual = 'true' then 'Real' 
                     else 'Virtual' 
                     end as is_actual
                     
FROM product_purchased_receipts
LEFT JOIN products ON products.id = product_purchased_receipts.product_id
LEFT JOIN packing_unit_products ON packing_unit_products.product_id = products.id
LEFT JOIN purchased_receipts ON purchased_receipts.id = product_purchased_receipts.purchased_receipt_id
LEFT JOIN purchased_receipt_statuses ON purchased_receipt_statuses.id = purchased_receipts.purchased_receipt_status_id
LEFT JOIN packing_units ON packing_units.id = product_purchased_receipts.packing_unit_id
LEFT JOIN product_units ON products.unit_id = product_units.id
LEFT JOIN suppliers ON suppliers.id = purchased_receipts.supplier_id
LEFT JOIN brands ON brands.id = products.brand_id
left join categories on categories.id = products.category_id
left join warehouses on warehouses.id = purchased_receipts.warehouse_id
WHERE product_purchased_receipts.purchased_item_count <> 0
      AND purchased_receipts.purchased_receipt_status_id IN (4,5,7)
      AND purchased_receipts.date::date >= current_date
    and product_purchased_receipts.final_price > 0 
     
     
    )
    SELECT product_id,
	sum(purchase_min_count) as all_day_qty ,
	coalesce(sum(case when hour between DATE_Part('hour',CURRENT_TIME) -1 and DATE_Part('hour',CURRENT_TIME) then purchase_min_count end),0) as last_hour_qty,
	avg(final_item_price/basic_unit_count) as item_price
    FROM prs
	group by 1
'''
prs_data = setup_environment_2.dwh_pg_query(query, columns = ['product_id','all_day_qty','last_hour_qty','item_price']) 
prs_data.product_id=pd.to_numeric(prs_data.product_id)
prs_data.all_day_qty=pd.to_numeric(prs_data.all_day_qty)
prs_data.last_hour_qty=pd.to_numeric(prs_data.last_hour_qty)
prs_data.item_price=pd.to_numeric(prs_data.item_price)

In [5]:
query = '''
SELECT
  t_product_id AS product_id,
  s_beg_stock AS av_STOCKS,
  p_purchased_item_count AS pr_qty,
  p_final_price AS pr_value,
  CASE
    WHEN p_purchased_item_count <> 0 THEN p_final_price / p_purchased_item_count
    ELSE 0
  END AS item_price
FROM
  finance.wac_tracker wt
  JOIN products ON products.id = wt.t_product_id 
  JOIN brands ON brands.id = products.brand_id
  JOIN categories c ON c.id = products.category_id
  JOIN product_units ON product_units.id = products.unit_id
  where wt.t_date::date = CURRENT_DATE
  and p_purchased_item_count > 0 
'''
try:
    reflected_in_wac = setup_environment_2.dwh_pg_query(query, columns = ['product_id','av_STOCKS','pr_qty','pr_value','cu_item_price']) 
    reflected_in_wac.product_id=pd.to_numeric(reflected_in_wac.product_id)
    reflected_in_wac.av_STOCKS=pd.to_numeric(reflected_in_wac.av_STOCKS)
    reflected_in_wac.pr_qty=pd.to_numeric(reflected_in_wac.pr_qty)
    reflected_in_wac.cu_item_price=pd.to_numeric(reflected_in_wac.cu_item_price)
    reflected_in_wac.pr_value=pd.to_numeric(reflected_in_wac.pr_value)
except:    
    columns = ['product_id','av_STOCKS','pr_qty','pr_value','cu_item_price']
    reflected_in_wac = pd.DataFrame(columns=columns)      

In [6]:
query = '''
select 
		f.product_id,
		CONCAT(products.name_ar,' ',products.size,' ',product_units.name_ar) as sku,
		brands.name_ar as brand, 
		categories.name_ar as cat,	
		f.wac1,
		f.wac_p
from finance.all_cogs f
JOIN products on products.id=f.product_id
JOIN brands on products.brand_id = brands.id 
JOIN categories ON products.category_id = categories.id
JOIN product_units ON product_units.id = products.unit_id 
where current_timestamp between f.from_date and f.to_date 
and not categories.name_ar in (
    SELECT categories.name_ar AS cat
                    FROM categories
                    JOIN sections s ON s.id = categories.section_id
                    WHERE   categories.name_ar LIKE '%سايب%'
                        OR  categories.name_ar LIKE '%بالتة%'
                        OR  categories.section_id IN (225, 318, 285, 121, 87, 351, 417)

)
'''

wacs = query_snowflake(query, columns = ['product_id','sku','brand','cat','wac1','wac_p']) 
wacs.product_id=pd.to_numeric(wacs.product_id)
wacs.wac1=pd.to_numeric(wacs.wac1)
wacs.wac_p=pd.to_numeric(wacs.wac_p)

In [7]:
query ='''

select region,cohort_id,product_id,packing_unit_id,basic_unit_count,price
from (
select   case when cpc.cohort_id in (700,695) then 'Cairo'
             when cpc.cohort_id in (701) then 'Giza'
             when cpc.cohort_id in (704,698) then 'Delta East'
             when cpc.cohort_id in (703,697) then 'Delta West'
             when cpc.cohort_id in (696,1123,1124,1125,1126) then 'Upper Egypt'
             when cpc.cohort_id in (702,699) then 'Alexandria'
        end as region,
		cohort_id,
        pu.product_id,
		pu.packing_unit_id as packing_unit_id,
		pu.basic_unit_count,
        cpc.price as price,
		cpc.created_at,
		max(cpc.created_at)over(partition by pu.product_id ,pu.packing_unit_id,cohort_id) as max_date 
from cohort_pricing_changes cpc 
join    PACKING_UNIT_PRODUCTS pu on pu.id = cpc.product_packing_unit_id
WHERE   cpc.cohort_id in (700,701,702,703,704,696,695,698,697,699,1123,1124,1125,1126)
)x 
where created_at = max_date
'''
prices = setup_environment_2.dwh_pg_query(query, columns = ['region','cohort_id','product_id','packing_unit_id','basic_unit_count','price'])
prices.product_id=pd.to_numeric(prices.product_id)
prices.cohort_id=pd.to_numeric(prices.cohort_id)
prices.packing_unit_id=pd.to_numeric(prices.packing_unit_id)
prices.basic_unit_count=pd.to_numeric(prices.basic_unit_count)
prices.price=pd.to_numeric(prices.price)

In [8]:
query = '''
select region,product_id,optimal_margin,coalesce(MIN_BOUNDARY,optimal_margin) as MIN_BOUNDARY
from (
select region,product_id,greatest(optimal_bm,coalesce(MIN_BOUNDARY,optimal_bm)) as optimal_margin,MIN_BOUNDARY,row_number()over(partition by region,product_id order by created_at desc) as rnk 
from materialized_views.PRODUCT_STATISTICS
qualify rnk = 1 
)
where optimal_margin is not null
'''
stats = query_snowflake(query, columns = ['region','product_id','optimal_margin','min_margin'])
stats.product_id=pd.to_numeric(stats.product_id)
stats.optimal_margin=pd.to_numeric(stats.optimal_margin)
stats.min_margin=pd.to_numeric(stats.min_margin)

In [9]:
query = '''
SELECT DISTINCT cat, brand, margin as target_bm
FROM    performance.commercial_targets cplan
QUALIFY CASE WHEN DATE_TRUNC('month', MAX(DATE)OVER()) = DATE_TRUNC('month', CURRENT_DATE) THEN DATE_TRUNC('month', CURRENT_DATE)
ELSE DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month') END = DATE_TRUNC('month', date)
'''
brand_cat_target  = query_snowflake(query, columns = ['cat','brand','target_bm'])
brand_cat_target.target_bm=pd.to_numeric(brand_cat_target.target_bm)

In [10]:
query = '''
select cat,sum(target_bm *(target_nmv/cat_total)) as cat_target_margin
from (
select *,sum(target_nmv)over(partition by cat) as cat_total
from (
select cat,brand,avg(target_bm) as target_bm , sum(target_nmv) as target_nmv
from (
SELECT DISTINCT date,city as region,cat, brand, margin as target_bm,nmv as target_nmv
FROM    performance.commercial_targets cplan
QUALIFY CASE WHEN DATE_TRUNC('month', MAX(DATE)OVER()) = DATE_TRUNC('month', CURRENT_DATE) THEN DATE_TRUNC('month', CURRENT_DATE)
ELSE DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month') END = DATE_TRUNC('month', date)
)
group by all
)
)
group by all 
'''
cat_target  = query_snowflake(query, columns = ['cat','cat_target_margin'])
cat_target.cat_target_margin=pd.to_numeric(cat_target.cat_target_margin)

In [11]:
query = '''
select * 
from materialized_views.negative_margin_exceptions
'''
exceptions = query_snowflake(query, columns = ['region','product_id','sku'])
exceptions.product_id=pd.to_numeric(exceptions.product_id)
exceptions=exceptions[['region','product_id']]
exceptions['exception']=1

cohort_def = {
    'region': ['Cairo', 'Giza', 'Alexandria', 'Delta East', 'Delta West','Upper Egypt','Upper Egypt','Upper Egypt','Upper Egypt'],
    'cohort_id': [700, 701, 702, 704, 703, 1123,1124,1125,1126]
}
region_cohort_map = pd.DataFrame(cohort_def)
exceptions = exceptions.merge(region_cohort_map,on='region')

In [12]:
query = '''
select  product_id ,sum(AVAILABLE_STOCK) as stocks
from materialized_views.stock_snap_shots_recent 
where date_trunc('hour',Timestamp)  =  date_trunc('hour',CURRENT_TIMESTAMP) - interval '1 hour'
and   warehouse_id not in (6,9,10)
group by all 
'''
stocks = query_snowflake(query, columns = ['product_id','stocks'])
stocks.product_id=pd.to_numeric(stocks.product_id)
stocks.stocks=pd.to_numeric(stocks.stocks)

In [13]:
merged_data = prs_data.merge(reflected_in_wac,on='product_id',how='left')
merged_data=merged_data.fillna(0)
final_data = wacs.merge(merged_data,on='product_id',how='left')
final_data = final_data.merge(stocks,on='product_id',how='left')
final_data = final_data.fillna(0)
final_data['av_STOCKS'] =final_data['stocks']
final_data['not_reflected_qty'] = final_data['all_day_qty'] - final_data['pr_qty']
final_data['not_reflected_qty'] = final_data['not_reflected_qty'].fillna(0)
final_data['new_wac1'] = (((final_data['av_STOCKS']+final_data['pr_qty'])*final_data['wac1'])+(final_data['not_reflected_qty']*final_data['item_price']))/(final_data['av_STOCKS']+final_data['pr_qty']+final_data['not_reflected_qty'])
final_data['wac1_change'] = (final_data['new_wac1']-final_data['wac1'])/final_data['wac1']
final_data['new_wac_p'] = final_data['wac_p'] *(1+final_data['wac1_change'])
final_data['new_wac_p'] = final_data['new_wac_p'].fillna(final_data['wac_p'])
final_data = final_data.merge(prices,on='product_id')
final_data['new_wac_p'] = final_data['new_wac_p']*final_data['basic_unit_count'] 
final_data['bm'] = (final_data['price']-final_data['new_wac_p'])/final_data['price']
final_data = final_data.merge(stats,on=['product_id','region'],how='left')
final_data = final_data.merge(brand_cat_target,on=['cat','brand'],how='left')
final_data = final_data.merge(cat_target,on=['cat'])
final_data['final_optimal'] = final_data['optimal_margin'].fillna(final_data["target_bm"]).fillna(final_data["cat_target_margin"])
final_data['final_min'] = final_data['min_margin'].fillna(final_data["target_bm"]*0.8).fillna(final_data["cat_target_margin"]*0.8)

cond = [
    
    ((final_data['av_STOCKS'].isna()) & (final_data['bm']<0)),
    ((~final_data['av_STOCKS'].isna()) & (final_data['bm']<final_data['final_min']*0.5)&(final_data['cat'] != 'حاجه ساقعه')),
    ((~final_data['av_STOCKS'].isna()) & (final_data['bm']<0)&(final_data['cat'] == 'حاجه ساقعه')),
    
] 
cho = [
    final_data['new_wac_p']/(1-final_data['final_optimal']) , 
    final_data['new_wac_p']/(1-final_data['final_optimal']),
    final_data['new_wac_p']/(1-final_data['final_optimal'])
    
]
final_data['final_new_price'] = np.select(cond,cho,np.nan)
final_data['final_new_price'] = np.round(final_data['final_new_price']/4)*4
final_data = final_data[~final_data['final_new_price'].isna()]
final_data = final_data[final_data['final_new_price']>=final_data['price']*1.005]

final_data = final_data[(final_data['stocks']> 0)|(final_data['all_day_qty']>0)]
final_data = final_data.merge(exceptions,on=['product_id','region','cohort_id'],how='left')
final_data=final_data[final_data['exception'].isna()]
final_data.sort_values(['cohort_id', 'product_id', 'basic_unit_count']).reset_index(drop=True)
final_data['ind'] = 1
final_data['ind'] = final_data.groupby(['cohort_id', 'product_id']).ind.cumsum()
remove_min_pu = pd.read_csv('skus_to_remove_min.csv')
remove_min_pu['remove_min'] = 1
final_data = final_data.merge(remove_min_pu[['product_id','remove_min']], on='product_id', how='left')

to_upload = final_data[['cohort_id','product_id','sku','packing_unit_id','final_new_price','region','ind','remove_min']]

In [22]:
to_upload = to_upload[to_upload['cohort_id']==700]

In [23]:
to_upload=to_upload[to_upload['product_id']==879]

In [24]:
to_upload

,cohort_id,product_id,sku,packing_unit_id,final_new_price,region,ind,remove_min
771,700,879,الطاووس خل أبيض طبيعي - delisted 900 مل,1,168.0,Cairo,1,NaN


In [ ]:
# to_upload = prices.copy()
# to_upload=to_upload.merge(wacs ,on ='product_id')
# to_upload = to_upload[(to_upload['product_id'].isin([11770]))]
# to_upload['final_new_price'] = np.maximum(np.round(to_upload['wac_p']*(to_upload['basic_unit_count'])/(1-0.055)),138)
# to_upload['ind'] =1 
# to_upload['remove_min'] = np.nan
# to_upload['margin'] = (to_upload['final_new_price']-(to_upload['wac_p']*to_upload['basic_unit_count']))/to_upload['final_new_price']
# to_upload=to_upload[to_upload['final_new_price'] > to_upload['price']]
# to_upload

In [28]:
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
from requests import get
from pathlib import Path
import requests
    
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            return get_secret_value_response['SecretString']
        else:
            return base64.b64decode(get_secret_value_response['SecretBinary'])

        
pricing_api_secret = json.loads(get_secret("prod/pricing/api/"))
username = pricing_api_secret["egypt_username"]
password = pricing_api_secret["egypt_password"]
secret = pricing_api_secret["egypt_secret"]

# get access token
def get_access_token(url, client_id, client_secret):
    """
    get_access_token function takes three parameters and returns a session token
    to connect to MaxAB APIs

    :param url: production MaxAB token URL
    :param client_id: client ID
    :param client_secret: client sercret
    :return: session token
    """
    response = requests.post(
        url,
        data={"grant_type": "password",
              "username": username,
              "password": password},
        auth=(client_id, client_secret),
    )
    return response.json()["access_token"]


def post_prices(id_,file_name):
    token = get_access_token('https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token',
                             'main-system-externals',
                             secret)
    url = "https://api.maxab.info/main-system/api/admin-portal/cohorts/{}/pricing".format(id_)
    payload={}
    files=[
      ('sheet',(file_name,open(file_name,'rb'),'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))
    ]
    headers = {
      'Authorization': 'bearer {}'.format(token)}

    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    return response

In [31]:
for cohort in to_upload.cohort_id.unique():
        upload = to_upload[to_upload['cohort_id']==cohort]
        out=upload[['product_id', 'sku', 'packing_unit_id', 'final_new_price', 'ind', 'remove_min']].copy()
        out.columns = ['Product ID','Product Name','Packing Unit ID','Price','ind','remove_min']
        out['Visibility (YES/NO)'] = 'YES'
        out.loc[(out['ind'] == 1) & (out['remove_min'] == 1), 'Visibility (YES/NO)'] = 'NO'
        out.drop(columns=['ind','remove_min'], inplace=True)
        out = out.drop_duplicates()
        out['Execute At (format:dd/mm/yyyy HH:mm)'] = None
        out['Tags'] = None
        file_name_ = 'Uploads/1_new_{}.xlsx'.format(cohort).replace(' ','_')
        out.to_excel(file_name_,index = False,engine = 'xlsxwriter')
        time.sleep(5)
        x = post_prices(cohort, 'Uploads/1_new_{}.xlsx'.format(cohort).replace(' ','_'))
        if ('"success":true' in str(x.content).lower()):
            print(f"Prices are upoladed successfuly cohort:{cohort}")
        else:
            print(f"ERROR cohort: {cohort}")
            print(x.content)
            break


ERROR cohort: 700
b'{"success":false,"message":"validation.500","details":[]}'


In [27]:
out.to_excel('check_data.xlsx')